In [1]:
import cv2 
import matplotlib.pyplot as plt
from matplotlib import animation
from PIL import Image
import time
from pycoral.adapters import common
from pycoral.adapters import segment
from pycoral.utils.edgetpu import make_interpreter
import numpy as np
import threading
%matplotlib inline

In [2]:
model_name = 'deeplabv3_mnv2_pascal_quant_edgetpu.tflite'

In [3]:
def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  indices = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((indices >> channel) & 1) << shift
    indices >>= 3

  return colormap

In [4]:
def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]

In [5]:
interpreter = make_interpreter(model_name, device=':0')
interpreter.allocate_tensors()
width, height = common.input_size(interpreter)

In [6]:
vid = cv2.VideoCapture(0)

out = cv2.VideoWriter('out.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, (640,480))
i = 0
while i< 500: 
    i+=1
    ret, vf = vid.read()
    if not ret:
        break
    frame = cv2.cvtColor(vf, cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(frame, (width, height), interpolation = cv2.INTER_LANCZOS4)
    common.set_input(interpreter, resized_img)
    interpreter.invoke()
    result = segment.get_output(interpreter)
    print(f'=>{i}')
    new_width, new_height = resized_img.shape[0:2]
    result = result[:new_height, :new_width]
    mask_img = Image.fromarray(label_to_color_image(result).astype(np.uint8))
    mask_img = mask_img.resize((frame.shape[1],frame.shape[0]), resample=Image.Resampling.LANCZOS)
    out.write(cv2.cvtColor(np.asarray(Image.blend(Image.fromarray( frame), mask_img, 0.4)),cv2.COLOR_RGB2BGR))
    time.sleep(0.02)

out.release()
vid.release() 
    

=>1
=>2
=>3
=>4
=>5
=>6
=>7
=>8
=>9
=>10
=>11
=>12
=>13
=>14
=>15
=>16
=>17
=>18
=>19
=>20
=>21
=>22
=>23
=>24
=>25
=>26
=>27
=>28
=>29
=>30
=>31
=>32
=>33
=>34
=>35
=>36
=>37
=>38
=>39
=>40
=>41
=>42
=>43
=>44
=>45
=>46
=>47
=>48
=>49
=>50
=>51
=>52
=>53
=>54
=>55
=>56
=>57
=>58
=>59
=>60
=>61
=>62
=>63
=>64
=>65
=>66
=>67
=>68
=>69
=>70
=>71
=>72
=>73
=>74
=>75
=>76
=>77
=>78
=>79
=>80
=>81
=>82
=>83
=>84
=>85
=>86
=>87
=>88
=>89
=>90
=>91
=>92
=>93
=>94
=>95
=>96
=>97
=>98
=>99
=>100
=>101
=>102
=>103
=>104
=>105
=>106
=>107
=>108
=>109
=>110
=>111
=>112
=>113
=>114
=>115
=>116
=>117
=>118
=>119
=>120
=>121
=>122
=>123
=>124
=>125
=>126
=>127
=>128
=>129
=>130
=>131
=>132
=>133
=>134
=>135
=>136
=>137
=>138
=>139
=>140
=>141
=>142
=>143
=>144
=>145
=>146
=>147
=>148
=>149
=>150
=>151
=>152
=>153
=>154
=>155
=>156
=>157
=>158
=>159
=>160
=>161
=>162
=>163
=>164
=>165
=>166
=>167
=>168
=>169
=>170
=>171
=>172
=>173
=>174
=>175
=>176
=>177
=>178
=>179
=>180
=>181
=>182
=>183
=>184
=>18